In [1]:
import pandas as pd
import graphviz as gv
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_input = pd.read_csv('train.csv', sep=',')
df_test  = pd.read_csv('test.csv', sep=',')

In [6]:
from libpgm.graphskeleton import GraphSkeleton
from libpgm.nodedata import NodeData

In [8]:
from libpgm.tablecpdfactorization import TableCPDFactorization

SyntaxError: invalid syntax (tablecpdfactor.py, line 235)

In [9]:
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork

SyntaxError: invalid syntax (discretebayesiannetwork.py, line 87)

In [10]:
df_train            = df_input[['Survived', 'Pclass','Sex', 'Fare']][df_input.Fare!=0].dropna()
df_train.dropna(inplace=True)
df_train.loc[:,'Sex']  = df_train.Sex.map({'female':0 , 'male':1})

# Fare is arbitrary divided in two categories: cheap and expensive
df_train.loc[:,'Fare'] = pd.cut(df_train.Fare, [df_input.Fare.min(),15, df_input.Fare.max()], labels =[0,1])
df_train = df_train.rename(columns = {'Survived' : 'Surv'})
df_train_target        = df_train['Surv']

In [11]:
def get_probs_surv_cond(df, df_target, Surv, Pclass, Sex):
    # Return survival probability conditioned on Class and Sex
    # P(Surv | Sex, Pclass)
    return (df[ (df.Surv==Surv) & (df.Pclass==Pclass) & (df.Sex==Sex)].shape[0]
            /(1.0*df[(df.Pclass==Pclass) & (df.Sex==Sex)].shape[0]))

def format_data(df):
    result = []
    for row in df.itertuples():
        #print(row.Pclass)
        result.append(dict(Surv= row.Surv, Class=row.Pclass , Sex=row.Sex, Fare=row.Fare ))
    return result

def calc_BNprob(df_test):
    
    result = pd.Series()
    
    for row in df_test.itertuples():
        tablecpd=TableCPDFactorization(bn)
        prob_surv = tablecpd.specificquery(dict(Surv='1'), dict(Fare=str(row.Fare) , Sex=str(row.Sex) , Class=str(row.Pclass) ))

        if prob_surv >= 0.5:
            surv_class = 1
        else:
            surv_class  = 0        
        result = result.append(pd.Series([surv_class]), ignore_index = True )
    return result

def calc_accuracy(dff_train, dff_train_target, nb_iterations):
    
    result = np.zeros(nb_iterations)

    for itera in range(nb_iterations):
        XX_train, XX_test, yy_train, yy_test = train_test_split(dff_train, dff_train_target, test_size=0.33)
        data4bn = format_data(XX_train)
        learner = PGMLearner()
        # estimate parameters
        result_bn = learner.discrete_mle_estimateparams(skel, data4bn)
        #result_bn.Vdata
        result_predict = calc_BNprob(XX_test)
        BN_test_probs = pd.DataFrame()
        BN_test_probs['ground_truth'] = yy_test
        Test_prob = pd.concat([yy_test.reset_index().Surv, result_predict],  axis = 1, ignore_index = True)\
                    .rename(columns = {0:'ground_truth' , 1:'class_resu'})
        accuracy = Test_prob[Test_prob.ground_truth == Test_prob.class_resu].shape[0]/(1.0*Test_prob.shape[0])
        #print("Accuracy is {}").format(accuracy)
        result[itera] = accuracy
        
    return result


In [12]:
print ("P(Class|Fare)")
print ("Class= 1, 2 ,3  ; Fare = 0 (cheap):")
print(df_train[(df_train.Fare==0) & (df_train.Pclass==1)].shape[0] /(1.0*df_train[df_train.Fare==0].shape[0]),
df_train[(df_train.Fare==0) & (df_train.Pclass==2)].shape[0] /(1.0*df_train[df_train.Fare==0].shape[0]),
df_train[(df_train.Fare==0) & (df_train.Pclass==3)].shape[0]/(1.0*df_train[df_train.Fare==0].shape[0]))
print ("Class= 1, 2 ,3  ; Fare = 1 (expensive):")
print(df_train[(df_train.Fare==1) & (df_train.Pclass==1)].shape[0] /(1.0*df_train[df_train.Fare==1].shape[0]),
df_train[(df_train.Fare==1) & (df_train.Pclass==2)].shape[0] /(1.0*df_train[df_train.Fare==1].shape[0]),
df_train[(df_train.Fare==1) & (df_train.Pclass==3)].shape[0]/(1.0*df_train[df_train.Fare==1].shape[0]))

#Sex: Prior probability
print("------------")
print ("P(Sex)")
print ("Sex = 0 (female), 1 (male)")
print (df_train[df_train.Sex==0].shape[0]/float(df_train.Sex.shape[0]) , 
       df_train[df_train.Sex==1].shape[0]/float(df_train.Sex.shape[0]))

# Surv Probability
print("------------")
print("P(Surv|Class,Sex)")
print("Surv = 0 ,1 , Class = 1 , Sex = 0")
print(get_probs_surv_cond(df_train, df_train_target, 0, 1, 0),
get_probs_surv_cond(df_train, df_train_target, 1, 1, 0))
print("Surv = 0 ,1 , Class = 2 , Sex = 0")
print(get_probs_surv_cond(df_train, df_train_target, 0, 2, 0),
get_probs_surv_cond(df_train, df_train_target, 1, 2, 0))
print("Surv = 0 ,1 , Class = 3 , Sex = 0")
print(get_probs_surv_cond(df_train, df_train_target, 0, 3, 0),
get_probs_surv_cond(df_train, df_train_target, 1, 3, 0))
print("Surv = 0 ,1 , Class = 1 , Sex = 1")
print(get_probs_surv_cond(df_train, df_train_target, 0, 1, 1),
get_probs_surv_cond(df_train, df_train_target, 1, 1, 1))
print("Surv = 0 ,1 , Class = 2 , Sex = 1")
print(get_probs_surv_cond(df_train, df_train_target, 0, 2, 1),
get_probs_surv_cond(df_train, df_train_target, 1, 2, 1))
print("Surv = 0 ,1 , Class = 3 , Sex = 1")
print(get_probs_surv_cond(df_train, df_train_target, 0, 3, 1),
get_probs_surv_cond(df_train, df_train_target, 1, 3, 1))

P(Class|Fare)
Class= 1, 2 ,3  ; Fare = 0 (cheap):
0.002257336343115124 0.20090293453724606 0.7968397291196389
Class= 1, 2 ,3  ; Fare = 1 (expensive):
0.48498845265588914 0.20554272517321015 0.3094688221709007
------------
P(Sex)
Sex = 0 (female), 1 (male)
0.3584474885844749 0.6415525114155252
------------
P(Surv|Class,Sex)
Surv = 0 ,1 , Class = 1 , Sex = 0
0.031914893617021274 0.9680851063829787
Surv = 0 ,1 , Class = 2 , Sex = 0
0.07894736842105263 0.9210526315789473
Surv = 0 ,1 , Class = 3 , Sex = 0
0.5 0.5
Surv = 0 ,1 , Class = 1 , Sex = 1
0.6153846153846154 0.38461538461538464
Surv = 0 ,1 , Class = 2 , Sex = 1
0.8333333333333334 0.16666666666666666
Surv = 0 ,1 , Class = 3 , Sex = 1
0.8658892128279884 0.13411078717201166


In [ ]:
df_train

In [13]:
nd       = NodeData()
skel     = GraphSkeleton()
jsonpath_skel ="titanic_skel.json"
jsonpath_node ="titanic_nodes.json"
nd.load(jsonpath_node)
skel.load(jsonpath_skel)

# load bayesian network
bn       = DiscreteBayesianNetwork(skel, nd)

AttributeError: 'NodeData' object has no attribute 'dictload'